In [1]:
import numpy as np
np.random.seed(13)
from sklearn import svm
import pandas as pd
import matplotlib.pyplot as plt
import math
from __future__ import division
from sklearn.model_selection import KFold
from sklearn.preprocessing import normalize
from stockstats import StockDataFrame 
from googlefinance.client import get_price_data, get_prices_data, get_prices_time_data
import dataPreparation
param = {
    'q': 'ADBE', # Stock symbol (ex: "AAPL")
    'i': "3600", # Interval size in seconds ("86400" = 1 day intervals)
    'x': "NASD", # INDEXNASDAQStock exchange symbol on which stock is traded (ex: "NASD")
    'p': "2Y" # Period (Ex: "1Y" = 1 year)
    }
   
# link to doc http://www.networkerror.org/component/content/article/1-technical-wootness/44-googles-undocumented-finance-api.html
# get price data (return pandas dataframe) 1465948800
priceData = get_price_data(param)
#print(priceData)
stock = StockDataFrame.retype(priceData)
#print(stock)
price = pd.DataFrame(stock)
#['macd','macds', 'boll_ub', 'boll_lb','rsi_6','rsi_12','vr_6_sma','wr_10','wr_6''close_10_sma''tr''middle_{}_sma'
#df['dma'] = df['close_10_sma'] - df['close_50_sma'] pdi'] = cls._get_pdi(df, 14)df['mdi'] = cls._get_mdi(df, 14)
#df['dx'] = cls._get_dx(df, 14) df['adx'] = df['dx_6_ema'] df['adxr'] = df['adx_6_ema']]
#
string=['open_-3_d','rsi_3']

#print(idxs)

stock.get(string)
price = stock
print(price)

                         open      high       low     close  volume  \
2016-05-02 22:00:00   94.2900   94.9800   93.8095   94.2900  167412   
2016-05-02 23:00:00   94.2800   94.7100   94.0100   94.1200  254881   
2016-05-03 00:00:00   94.1600   94.4300   94.0400   94.3100  168646   
2016-05-03 01:00:00   94.2300   94.7000   94.1300   94.6300  190779   
2016-05-03 02:00:00   94.6250   94.8600   94.4990   94.6600  184986   
2016-05-03 03:00:00   94.6900   94.9900   94.6000   94.9275  211450   
2016-05-03 04:00:00   94.9300   94.9900   94.7650   94.9000  751356   
2016-05-03 22:00:00   94.0700   94.3899   93.8100   93.8900  230060   
2016-05-03 23:00:00   93.8800   93.8800   93.2800   93.4800  185040   
2016-05-04 00:00:00   93.4800   93.7100   93.0900   93.3400  156469   
2016-05-04 01:00:00   93.3700   93.8000   93.1500   93.7850  115336   
2016-05-04 02:00:00   93.7400   94.2100   93.7000   93.9600  124905   
2016-05-04 03:00:00   93.9700   94.0300   93.5600   93.7000  144897   
2016-0

/home/andrea/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/home/andrea/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [2]:


xtemp=price[string]
xtemp=np.nan_to_num(np.asarray(xtemp, dtype=float))
xtemp=normalize(xtemp,axis=0,norm='max')
print(len(xtemp))
accintime=[]
trendwindowtime=[46,70,100]
for t in trendwindowtime:
    #1
#label because of the maket and append values without data
#simo theroy past trend
    x=[]
    y=[]
    print('============================================================')
    print('============================================================')
    print('Working on window:',t)
    ##QUI C E L'UNICO APPUNTO GUARDA SE CON +1 CAMBIA
    for i in range(0,len(price)-t-1):
        y.append(np.sign(price.iloc[i+t+1]['close']-price.iloc[i+1]['open']))
        x.append(xtemp[i])
    y=np.array(y)
    x=np.array(x)
    #Split betwwen train-validation and test
    #Faccio random permuatation cosi e come se lo facessi su 5 dataset completamentediversi
    
    
    permindex=range(0,len(x))
    #permindex=np.random.permutation(permindex)
   
    train=0.8
    nt=math.ceil(len(x)*train)
    
    trainvalindex=permindex[0:nt]
    

    testindex=permindex[nt:]
    
    x_tv=[]
    y_tv=[]
    x_test=[]
    y_test=[]
    x_tv=x[trainvalindex]
    y_tv=y[trainvalindex]
    x_test=x[testindex]
    y_test=y[testindex]

    #Fairly sampling the tv
    posindex=np.where( y_tv > 0 )
    negindex=np.where( y_tv < 0 )

    yindex=[]
    nindex=min(len(posindex[0]),len(negindex[0]))

    #for i in range(1,nindex):
    y_tvnew=np.concatenate((y_tv[posindex[0][0:nindex]],y_tv[negindex[0][0:nindex]]))
    x_tvnew=np.concatenate((x_tv[posindex[0][0:nindex]],x_tv[negindex[0][0:nindex]]))

    #Fairly sampling the test 50% 50%
    posindex=np.where( y_test > 0 )
    negindex=np.where( y_test < 0 )

    yindex=[]
    nindex=min(len(posindex[0]),len(negindex[0]))

    #for i in range(1,nindex):
    y_testnew=np.concatenate((y_test[posindex[0][0:nindex]],y_test[negindex[0][0:nindex]]))
    x_testnew=np.concatenate((x_test[posindex[0][0:nindex]],x_test[negindex[0][0:nindex]]))
    
    nfold=10
    kf = KFold(n_splits=nfold, random_state=13, shuffle=True)


    cspace=np.logspace(-2,4,5)
    gspace=np.logspace(-4,4,5)
    bestsvm=None
    maxacc=0
    cvacc=0
    maxg=0
    maxc=0
    print('Model Selection...')
    #model selection
    for c in cspace:
        #print()
        for g in gspace:
            cvacc=0
            #crossvalidation
            for train_index, test_index in kf.split(x_tvnew):
                x_train, x_val = x_tvnew[train_index], x_tvnew[test_index]
                y_train, y_val = y_tvnew[train_index], y_tvnew[test_index]
                rbf_svm=svm.SVC(kernel='rbf',C=c,gamma=g)
                rbf_svm.fit(x_train,y_train)
                accuracy=rbf_svm.score(x_val,y_val)
                cvacc=cvacc+accuracy/nfold
            #print('ecco c and gamma',c,g)
            #print(cvacc)
            if(cvacc>maxacc):
                maxacc=cvacc
                maxg=g
                maxc=c
                bestsvm=rbf_svm
    
    print('ACCURACY on val:',maxacc)
    print('Gamma:',maxg)
    print('C:',maxc)
    #print(bestsvm)
    rbf_svm=svm.SVC(kernel='rbf',C=maxc,gamma=maxg)
    rbf_svm.fit(x_tvnew,y_tvnew)
    testacc=rbf_svm.score(x_testnew,y_testnew)
    print('Accuracy on TEST set:',testacc)

3517
Working on window: 46
Model Selection...
ACCURACY on val: 0.5229364132822798
Gamma: 0.0001
C: 10000.0
Accuracy on TEST set: 0.4595744680851064
Working on window: 70
Model Selection...
ACCURACY on val: 0.536734999251833
Gamma: 1.0
C: 316.22776601683796
Accuracy on TEST set: 0.5398230088495575
Working on window: 100
Model Selection...
ACCURACY on val: 0.5439773264052905
Gamma: 1.0
C: 316.22776601683796
Accuracy on TEST set: 0.539906103286385


In [3]:
3517
============================================================
============================================================
Working on window: 2
Model Selection...
ACCURACY on val: 0.5389458040852464
Gamma: 0.01
C: 316.22776601683796
Accuracy on TEST set: 0.47770700636942676
============================================================
============================================================
Working on window: 6
Model Selection...
ACCURACY on val: 0.5587177286705286
Gamma: 0.01
C: 10.0
Accuracy on TEST set: 0.5050167224080268
============================================================
============================================================
Working on window: 12
Model Selection...
ACCURACY on val: 0.5629764573991032
Gamma: 100.0
C: 10.0
Accuracy on TEST set: 0.539622641509434
============================================================
============================================================
Working on window: 30
Model Selection...
ACCURACY on val: 0.5208872889257226
Gamma: 100.0
C: 10.0
Accuracy on TEST set: 0.5527426160337553

    
Ccompare this appl with adbe running
============================================================
============================================================
Working on window: 2
Model Selection...
ACCURACY on val: 0.5293182016079877
Gamma: 100.0
C: 10.0
Accuracy on TEST set: 0.5014749262536873
============================================================
============================================================
Working on window: 6
Model Selection...
ACCURACY on val: 0.5196169287387086
Gamma: 100.0
C: 316.22776601683796
Accuracy on TEST set: 0.5059171597633136
============================================================
============================================================
Working on window: 12
Model Selection...
ACCURACY on val: 0.5350872488021169
Gamma: 100.0
C: 316.22776601683796
Accuracy on TEST set: 0.5258358662613982
============================================================
============================================================
Working on window: 30
Model Selection...
ACCURACY on val: 0.54472128187768
Gamma: 100.0
C: 316.22776601683796
Accuracy on TEST set: 0.48575949367088606


SyntaxError: invalid syntax (<ipython-input-3-eeff514d9ab1>, line 2)

In [ ]:
import numpy as np
np.random.seed(13)
from sklearn import svm
import pandas as pd
import matplotlib.pyplot as plt
import math
from __future__ import division
from sklearn.model_selection import KFold
ticker='AAPL'
y=[]
price=pd.read_csv('/home/andrea/Desktop/NLFF/indexesLabeledNew/indexes'+ticker+'.csv')
#print(price)
trendwindowtime=[1,4,10,20,30]
#xtemp=price[['open','close','volume','macd','macds', 'boll_ub', 'boll_lb','rsi_6','rsi_12','vr_6_sma','wr_10','wr_6']]
price=price.iloc[2000:]
xtemp=price[['rsi_6']]
xtemp=np.nan_to_num(np.asarray(xtemp, dtype=float))
accintime=[]
for t in trendwindowtime:
    #1
#label because of the maket and append values without data
#simo theroy past trend
    x=[]
    y=[]
    print('============================================================')
    print('============================================================')
    print('Working on window:',t)
    for i in range(0,len(price)-t):
        y.append(np.sign(price.iloc[i+t]['close']-price.iloc[i]['open']))
        x.append(xtemp[i])
    y=np.array(y)
    x=np.array(x)
    #Split betwwen train-validation and test
    train=0.8
    nt=math.ceil(len(x)*train)
    x_tv=[]
    y_tv=[]
    x_test=[]
    y_test=[]
    x_tv=x[:nt]
    y_tv=y[:nt]
    x_test=x[nt:]
    y_test=y[nt:]
    #Fairly sampling the tv
    posindex=np.where( y_tv > 0 )
    negindex=np.where( y_tv < 0 )
    
    yindex=[]
    nindex=min(len(posindex[0]),len(negindex[0]))
    
    #for i in range(1,nindex):
    y_tvnew=np.concatenate((y_tv[posindex[0][0:nindex]],y_tv[negindex[0][0:nindex]]))
    x_tvnew=np.concatenate((x_tv[posindex[0][0:nindex]],x_tv[negindex[0][0:nindex]]))
    
    #Fairly sampling the test 50% 50%
    posindex=np.where( y_test > 0 )
    negindex=np.where( y_test < 0 )
    
    yindex=[]
    nindex=min(len(posindex[0]),len(negindex[0]))
    
    #for i in range(1,nindex):
    y_testnew=np.concatenate((y_test[posindex[0][0:nindex]],y_test[negindex[0][0:nindex]]))
    x_testnew=np.concatenate((x_test[posindex[0][0:nindex]],x_test[negindex[0][0:nindex]]))
    
    nfold=10
    kf = KFold(n_splits=nfold, random_state=13, shuffle=True)


    cspace=np.logspace(-2,4,5)
    gspace=np.logspace(-4,4,5)
    bestsvm=None
    maxacc=0
    cvacc=0
    maxg=0
    maxc=0
    print('Model Selection...')
    #model selection
    for c in cspace:
        print()
        for g in gspace:
            cvacc=0
            #crossvalidation
            for train_index, test_index in kf.split(x_tvnew):
                x_train, x_val = x_tvnew[train_index], x_tvnew[test_index]
                y_train, y_val = y_tvnew[train_index], y_tvnew[test_index]
                rbf_svm=svm.SVC(kernel='rbf',C=c,gamma=g)
                rbf_svm.fit(x_train,y_train)
                accuracy=rbf_svm.score(x_val,y_val)
                cvacc=cvacc+accuracy/nfold
            #print('ecco c and gamma',c,g)
            #print(cvacc)
            if(cvacc>maxacc):
                maxacc=cvacc
                maxg=g
                maxc=c
                bestsvm=rbf_svm
    
    print('ACCURACY on val:',maxacc)
    print('Gamma:',maxg)
    print('C:',maxc)
    #print(bestsvm)
    rbf_svm=svm.SVC(kernel='rbf',C=maxc,gamma=maxg)
    rbf_svm.fit(x_tvnew,y_tvnew)
    testacc=rbf_svm.score(x_testnew,y_testnew)
    print('Accuracy on TEST set:',testacc)
    print('LOOK WHAT APPENS IN TIME.....')

    accintime.append(testacc)
print(accintime)
    
    


In [ ]:
from stockstats import StockDataFrame 
from googlefinance.client import get_price_data, get_prices_data, get_prices_time_data
param = {
    'q': 'AAPL', # Stock symbol (ex: "AAPL")
    'i': "3600", # Interval size in seconds ("86400" = 1 day intervals)
    'x': "NASD", # INDEXNASDAQStock exchange symbol on which stock is traded (ex: "NASD")
    'p': "2Y" # Period (Ex: "1Y" = 1 year)
    }
   
# link to doc http://www.networkerror.org/component/content/article/1-technical-wootness/44-googles-undocumented-finance-api.html
# get price data (return pandas dataframe) 1465948800
priceData = get_price_data(param)
#print(priceData)
stock = StockDataFrame.retype(priceData)
#print(stock)
toSave = pd.DataFrame(stock)
idxs = np.array(['macd','macds', 'boll_ub', 'boll_lb','rsi_6','rsi_12','vr_6_sma','wr_10','wr_6'])
#print(idxs)
for index,x in np.ndenumerate(idxs):
#print(x)
    toSave = pd.concat([toSave, pd.DataFrame(stock.get(x))], axis=1)

print(toSave)
    #dat1 = pd.DataFrame({'dat1': [9,5]})
    #toSave.join(pd.DataFrame({x:stock.get(x)}))
    #print("date"+toSave)

In [ ]:
rbf_svm=svm.SVC(kernel='rbf',C=maxc,gamma=maxg)
rbf_svm.fit(x_tvnew,y_tvnew)
testacc=rbf_svm.score(x_testnew,y_testnew)
print('Accuracy on TEST set:',testacc)
print('LOOK WHAT APPENS IN TIME.....')